In [ ]:
import io
import os
from pathlib import Path

from dotenv import load_dotenv
from O365 import Account

load_dotenv(Path("~").expanduser() / "O365.env")

In [ ]:
def getO365Client():
    app_principal = {
        "client_id": os.environ["O365_client_id"],
        "client_secret": os.environ["O365_client_secret"],
        "tenant_id": os.environ["O365_tenant_id"],
    }
    auth = (app_principal["client_id"], app_principal["client_secret"])
    account = Account(
        auth, auth_flow_type="credentials", tenant_id=app_principal["tenant_id"]
    )
    if account.authenticate(scope=[]):
        return account
    return None


account = getO365Client()

In [ ]:
dir(account)

In [ ]:
site = account.sharepoint().search_site("Purchasing Bids")[0]

In [ ]:
library = site.list_document_libraries()[0]
library

In [ ]:
root = list(library.get_items())
root

In [ ]:
allowed = [
    "Purchasing Bids Owners",
    "Purchasing Bids Visitors",
    "Purchasing Bids Members",
    None,
]
permissionsToRemove = []

for folder in root:
    if "RFPs" in folder.name:

        for permission in folder.get_permissions():
            if permission.granted_to not in allowed:
                print(folder, permission.roles, permission.granted_to)
                # permission.delete()
                permissionsToRemove.append(permission)
        for subfolder in folder.get_items():
            for permission in subfolder.get_permissions():
                if permission.granted_to not in allowed:
                    print(subfolder, permission.roles, permission.granted_to)
                    # permission.delete()
                    permissionsToRemove.append(permission)

In [ ]:
for folder in root:
    if "RFPs" in folder.name:
        print(folder)
        if folder.shared is None:
            link = folder.share_with_link()
            print(link.share_link)
        for subfolder in folder.get_items():
            if subfolder.shared is None:
                link = subfolder.share_with_link()
                print(link.share_link)